In [1]:
from __future__ import division
import matplotlib 
# matplotlib.use('Qt5Agg') - интерактивные графики в отдельном окне \
# с использованием Qt

%matplotlib qt 
# интерактивные графики в отдельном окне (другой способ)


import matplotlib.pyplot as plt
import numpy as np
import time
import numba as nb
import math as mt
from scipy import interpolate as intp
import sys
from timeit import default_timer as timer

matplotlib.rcParams.update({'font.size': 25})

In [4]:
#@nb.jit(nopython=True)
def CalculateNorm(analiticalSolutionFunc, numericalSolutionArr, T, dt, amplitude):
    # calculating RRHS error
    numerator, denumenator = 0, 0
    tArray = np.arange(0, T + dt, dt)
    
    deltaArray = np.fabs(analiticalSolutionFunc(tArray) - numericalSolutionArr[:])
    numerator = np.sum(deltaArray**2)
    denumenator = np.sum((analiticalSolutionFunc(tArray))**2)

    RRHSerror = np.sqrt(numerator / denumenator)

    # calculating max|o| error
    maxModError = np.amax(deltaArray)
    errorsList = [100 * RRHSerror, 100*maxModError / amplitude]

    # allowing spline-interpolation to happen when measuring errors
    for i in range(len(errorsList)):
        if np.isnan(errorsList[i]) == True:
            errorsList[i] = sys.float_info.max
    
    return errorsList

In [5]:
#@nb.jit(nopython=True)
def eps(u, v):
    eps0 = 0.002
    mu1 = 0.2
    mu2 = 0.3
    return eps0 + (mu1*v) / (u + mu2)

#@nb.jit(nopython=True)
def rhsU(u, v, k, iStim=0.1):
    return -k*u*(u - a)*(u - 1) - u*v + iStim

#@nb.jit(nopython=True)
def rhsV(u, v, k):
    return eps(u, v)*(-v - k*u*(u - a - 1))


#@nb.jit((nb.float64[:], nb.int64, nb.float64, nb.int64, \
#                        nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], \
#                        nb.float64[:], nb.float64, nb.float64, nb.float64), nopython=True)
def CalculateUsingExplicitEuler(time_array, size, dt, counter, u, v, RhsU, RhsV, I_s, k,\
                                u_rest, v_rest):
    u[0] = u_rest
    v[0] = v_rest

    for i in range(1, size):
        #print('EE step: complete %.2f' % (i/float(size - 1)))
        RhsU[i - 1] = rhsU(u[i - 1], v[i - 1], k)
        RhsV[i - 1] = rhsV(u[i - 1], v[i - 1], k)

        u[i] = u[i - 1] + dt*RhsU[i - 1]
        v[i] = v[i - 1] + dt*RhsV[i - 1]



#@nb.jit((nb.float64[:], nb.int64, nb.float64, nb.int64, \
#                        nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:],\
#                        nb.float64[:], nb.float64, nb.float64, nb.float64), nopython=True)
def CalculateUsingSImplicitEuler(time_array, size, dt, counter, u, v, RhsU, RhsV, I_s, k,\
                                 u_rest, v_rest):

    u[0] = u_rest
    v[0] = v_rest
    stepForDer = 1e-2
    
    for i in range(1, size):
        #print('SIE step: complete %.2f' % (i/float(size - 1)))
        RhsU[i - 1] = rhsU(u[i - 1], v[i - 1], k)
        RhsV[i - 1] = rhsV(u[i - 1], v[i - 1], k)

        dRhsUdU = (rhsU(u[i - 1] + stepForDer, v[i - 1], k) \
                                - RhsU[i - 1]) / stepForDer

        v[i] = v[i - 1] + dt*RhsV[i - 1]
        u[i] = u[i - 1] + dt*RhsU[i - 1] / (1 - dt*dRhsUdU)

In [6]:
kOriginal = 8.
kArray = [float(i) for i in range(7, 13)]
a = 0.15

u_rest = 0.
v_rest = 0.

T = 40.
TScaled = 12.9*T
numBlocksAnalitical = 2**19
dtAnalitical = T/numBlocksAnalitical
numBlocksArray = np.asarray([int(2**n) for n in range(6, 18)])
dtArray = np.asarray([dtAnalitical] + list(float(T)/numBlocksArray[::-1]))
dtArrayScaled = 12.9*dtArray

print(dtArrayScaled)

[9.84191895e-04 3.93676758e-03 7.87353516e-03 1.57470703e-02
 3.14941406e-02 6.29882812e-02 1.25976562e-01 2.51953125e-01
 5.03906250e-01 1.00781250e+00 2.01562500e+00 4.03125000e+00
 8.06250000e+00]


#Функция Main, для дальнейшего расчета погрешностей разных методов по разным нормам

In [7]:
speedup = []
timingEE, timingSIE = [], []
error, error_e = [], []
listOfTimeArrays, listOfNumericalSolutions = list(), list()
analiticalSolutionFunc = None
MIN, MAX = 0., 0.

counter = 0
for dt in dtArray:
    print ('Iteration #%d' % counter)
    
    time_array = np.arange(0, T + dt, dt)
    time_array_scaled = 12.9 * time_array
    dtScaled = 12.9*dt
    SIZE = len(time_array)
    
    u = np.zeros(SIZE)
    v = np.ones(SIZE)
    RhsU = np.zeros(SIZE) # [0. for i in range(size)]
    RhsV = np.zeros(SIZE)

    u_e = np.zeros(SIZE)
    v_e = np.zeros(SIZE) #[0. for i in range(size)]

    RhsU_e = np.zeros(SIZE) #[0. for i in range(size)]
    RhsV_e = np.zeros(SIZE)

    I_s =10*np.ones(SIZE) #[10. for i in range(size)]



    NUM_LAUNCHES = 1 # N = 3 - used for calculaions for the article
    startEE = time.clock()
    for i in range(NUM_LAUNCHES):
        CalculateUsingExplicitEuler(time_array, SIZE, dt, counter, u_e, v_e, RhsU_e, RhsV_e, I_s, \
                                                                        kOriginal, u_rest, v_rest)
        #print('EE calculation #%d complete' % i)
    timingEE.append((time.clock() - startEE)/NUM_LAUNCHES)

    startSIE = time.clock()
    for i in range(NUM_LAUNCHES):
        CalculateUsingSImplicitEuler(time_array, SIZE, dt, counter, u, v, RhsU, RhsV, I_s, \
                                                                        kOriginal, u_rest, v_rest)
        #print('SIE calculation #%d complete' % i)   
    timingSIE.append((time.clock() - startSIE)/NUM_LAUNCHES)
      

    if (counter == 0):
        analiticalSolution = np.zeros(SIZE)
        analiticalSolution[:] = 100*u_e[:] - 80 # scaled to membrane voltage
        MIN, MAX = np.amin(analiticalSolution), np.amax(analiticalSolution)
        amplitude = MAX - MIN
        analiticalSolution[:] -= MIN
        analiticalSolutionFunc = intp.interp1d(time_array_scaled, analiticalSolution)


    u_e[:] = 100*u_e[:] - 80 # scaled to membrane voltage
    u[:] = 100*u[:] - 80 # scaled to membrane voltage

    u[:] -= MIN
    u_e[:] -= MIN


    errTmp = CalculateNorm(analiticalSolutionFunc, u, TScaled, dtScaled, amplitude)
    errTmp_e = CalculateNorm(analiticalSolutionFunc, u_e, TScaled, dtScaled, amplitude)

    error.append(errTmp)
    error_e.append(errTmp_e)

    listOfTimeArrays.append(time_array_scaled)
    listOfNumericalSolutions.append(u_e)
    counter += 1

Iteration #0


Iteration #1


Iteration #2


Iteration #3


Iteration #4
Iteration #5


Iteration #6
Iteration #7
Iteration #8
Iteration #9
Iteration #10
Iteration #11
Iteration #12


/Users/quothewhite/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/quothewhite/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


#Оценка погрешностей, таблица погрешностей

In [8]:
import pandas as pd

error = np.asarray(error)
error_e = np.asarray(error_e)

print(dtArray)

listOfTolerances = [1., 3., 5.]
LEN = len(listOfTolerances)
NUM_ERROR_TYPES = 2

errorFunctionRRMS = intp.interp1d(error[:,0], dtArrayScaled)
errorFunctionMaxmod = intp.interp1d(error[:,1], dtArrayScaled)
timingFunctionRRMS = intp.interp1d(error[:,0], timingSIE)
timingFunctionMaxmod = intp.interp1d(error[:,1], timingSIE)

error_e_FunctionRRMS = intp.interp1d(error_e[:,0], dtArrayScaled)
error_e_FunctionMaxmod = intp.interp1d(error_e[:,1], dtArrayScaled)
timing_e_FunctionRRMS = intp.interp1d(error_e[:,0], timingEE)
timing_e_FunctionMaxmod = intp.interp1d(error_e[:,1], timingEE)



print(error_e[:,0])
#print(errorFunctionRRMS(listOfTolerances) + 
 #                               errorFunctionMaxmod(listOfTolerances))

d = {('dt', 'Явный метод'): pd.Series(list(error_e_FunctionRRMS(listOfTolerances)) + 
                                list(error_e_FunctionMaxmod(listOfTolerances)),
                            index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]]),
     ('dt', 'Неявный метод'): pd.Series(list(errorFunctionRRMS(listOfTolerances)) + 
                            list(errorFunctionMaxmod(listOfTolerances)),
                            index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]]),
     ('t_calc', 'Явный метод'): pd.Series(list(timing_e_FunctionRRMS(listOfTolerances)) + 
                                list(timing_e_FunctionMaxmod(listOfTolerances)),
                            index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]]),
     ('t_calc', 'Неявный метод'): pd.Series(list(timingFunctionRRMS(listOfTolerances)) + 
                            list(timingFunctionMaxmod(listOfTolerances)),
                            index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]])
   }

micolumns = pd.MultiIndex.from_tuples([('dt', 'Явный метод'), 
                                       ('dt', 'Неявный метод'),
                                       ('t_calc', 'Явный метод'),
                                       ('t_calc', 'Неявный метод')
                                        ])


df = pd.DataFrame(d)
#df = pd.DataFrame(np.random.rand(6, 4),
#                  index=[3*['RRMS'] + 3*['Maxmod'], 2*[1, 3, 5]],
#                  columns=micolumns)

#df.unstack()
print(df)

writer = pd.ExcelWriter('ap_model_k_8.xlsx')
df.to_excel(writer, 'Sheet1')
writer.save()

[7.62939453e-05 3.05175781e-04 6.10351562e-04 1.22070312e-03
 2.44140625e-03 4.88281250e-03 9.76562500e-03 1.95312500e-02
 3.90625000e-02 7.81250000e-02 1.56250000e-01 3.12500000e-01
 6.25000000e-01]
[0.00000000e+000 6.02890702e-003 1.40674918e-002 3.01448005e-002
 6.22999385e-002 1.26612004e-001 2.55240941e-001 5.12499148e-001
 1.02686433e+000 2.05375239e+000 4.09003580e+000 9.32828516e+000
 1.79769313e+308]


                    dt                    t_calc            
         Неявный метод Явный метод Неявный метод Явный метод
RRMS   1      0.177235    0.490747      0.022909    0.004640
       3      0.527807    1.476136      0.006454    0.001617
       5      0.876175    2.365770      0.004142    0.000950
Maxmod 1      0.052388    0.113297      0.077329    0.022164
       3      0.155919    0.341746      0.025078    0.007380
       5      0.259013    0.575357      0.015060    0.004075


In [11]:
matplotlib.rcParams.update({'font.size': 25})

plt.figure(figsize=(13,8.5))
plt.loglog((timingSIE[1:]), (error[1:,0]), 'k-o', label='Упрощенный неявный метод', linewidth=4, markersize = 15)
plt.loglog((timingEE[1:]), (error_e[1:,0]), 'k-^', label='Явный метод', linewidth=4, markersize = 15)
plt.grid('on')
plt.xlabel('Время вычисления, с')
plt.ylabel('Погрешность RRMS, %')
plt.ylim([1e-3, 1e3])
plt.axhline(y=5, linewidth=4, color='k', linestyle='--', label='5%')
plt.axhline(y=1, linewidth=4, color='k', linestyle='-', label='1%')
plt.legend(loc='best', fontsize=20)

plt.figure(figsize=(13,8.5))
plt.loglog((timingSIE[1:]), (error[1:,1]), 'k-o', label='Упрощенный неявный метод', linewidth=4, markersize = 15)
plt.loglog((timingEE[1:]), (error_e[1:,1]), 'k-^', label='Явный метод', linewidth=4, markersize = 15)
plt.grid('on')
plt.xlabel('Время вычисления, с')
plt.ylabel('Погрешность Maxmod, %')
plt.ylim([1e-3, 1e3])
plt.axhline(y=5, linewidth=4, color='k', linestyle='--', label='5%')
plt.axhline(y=1, linewidth=4, color='k', linestyle='-', label='1%')
plt.legend(loc='best', fontsize=20)

/Users/quothewhite/anaconda/envs/python3/lib/python3.6/site-packages/numpy/ma/core.py:6641: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)


#Оценка шагов по времени, при которых достигается заданная точность

In [165]:
tolerance2 = 5.
dtArrayForTolerance = [float(errorFunctionRRMS(tolerance2))/12.9, float(errorFunctionMaxmod(tolerance2))/12.9]
print(dtArrayForTolerance)
#print(12.9*dtAnalitical)

[0.06792057269425687, 0.02007851911299917]


In [166]:
listOfTimeArrays, listOfNumericalSolutions = list(), list()
analiticalSolution = None
dtArray2 = [dtAnalitical] + dtArrayForTolerance
counter = 0
for dt in dtArray2:
    print ('Iteration #%d' % counter)
    
    time_array = np.arange(0, T + dt, dt)
    time_array_scaled = 12.9 * time_array
    dtScaled = 12.9*dt
    SIZE = len(time_array)
    
    u = np.zeros(SIZE)
    v = np.ones(SIZE)
    RhsU = np.zeros(SIZE) # [0. for i in range(size)]
    RhsV = np.zeros(SIZE)

    u_e = np.zeros(SIZE)
    v_e = np.zeros(SIZE) #[0. for i in range(size)]

    RhsU_e = np.zeros(SIZE) #[0. for i in range(size)]
    RhsV_e = np.zeros(SIZE)

    
    CalculateUsingExplicitEuler(time_array, SIZE, dt, counter, u_e, v_e, RhsU_e, RhsV_e, I_s, \
                                                                        kOriginal, u_rest, v_rest)     
    
    CalculateUsingSImplicitEuler(time_array, SIZE, dt, counter, u, v, RhsU, RhsV, I_s, \
                                                                kOriginal, u_rest, v_rest)
      
    if (counter == 0):
        analiticalSolution = np.zeros(SIZE)
        analiticalSolution[:] = 100*u_e[:] - 80 # scaled to membrane voltage
        MIN, MAX = np.amin(analiticalSolution), np.amax(analiticalSolution)
        amplitude = MAX - MIN
        analiticalSolution[:] -= MIN
        analiticalSolutionFunc = intp.interp1d(time_array_scaled, analiticalSolution)


    u[:] = 100*u[:] - 80 # scaled to membrane voltage
    u[:] -= MIN

    listOfTimeArrays.append(time_array_scaled)
    listOfNumericalSolutions.append(u)
    
    counter += 1

Iteration #0


Iteration #1
Iteration #2


In [167]:
plt.figure(figsize=(13, 8.5))
counter2 = 0

plt.plot(listOfTimeArrays[0], analiticalSolution, 'k--', label='Референсное решение', linewidth=2)
plt.plot(listOfTimeArrays[1], listOfNumericalSolutions[1], '-', label='Погрешность 5% по норме RRMS', linewidth=2)    
plt.plot(listOfTimeArrays[2], listOfNumericalSolutions[2], '-', label='Погрешность 5% по норме Maxmod', linewidth=2)
plt.legend(loc='best', prop={'size': 20})
plt.xlabel('Время, мс')
plt.ylabel('V, мВ')
plt.ylim([-10, 140])
plt.xlim([0, 516])
plt.grid('on')
plt.show()

#График аналитического решения

In [170]:
matplotlib.rcParams.update({'font.size': 25})

plt.figure(figsize=(13, 8.5))
plt.plot(listOfTimeArrays[0], analiticalSolution, 'k-', label='k = 8', linewidth=4)
plt.legend(loc='best', prop={'size': 20})
plt.xlabel('Время, мс')
plt.ylabel('V, мВ')
#plt.ylim([0, 140])
plt.xlim([0, 516])
plt.grid('on')
plt.show()

#Производные высших порядков и их Max значения

In [171]:
VAnalyticalArray = analiticalSolution

tAnalyticalArray = listOfTimeArrays[0] # corresponding array of timesteps, in msec
dt = np.diff(tAnalyticalArray)[0] # [0]=[1]=[2]=...=[n] because the grid is equally spaced


matplotlib.rcParams.update({'font.size': 20})
xTicks = [0, 250, 500]
#from matplotlib.ticker import FormatStrFormatter
# plotting derivatives calculated via finite-difference formulas
maxOrderDerivative = 5
valuesArray, tArray, dt = VAnalyticalArray, tAnalyticalArray, np.diff(tAnalyticalArray)[0]
plt.figure(figsize=(13, 8.5))

# plotting analytical solution
ax = plt.subplot(2, 3, 1)
plt.plot(tArray[1:], valuesArray[1:], 'k-', lw=3)  
plt.grid('on')
plt.xlim([0, 516])
ax.set_xticks(xTicks)
ax.set_xticklabels([])
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
plt.ylabel(r'$\mathregular{V, мВ}$')   

# plotting derivatives
for i in range(1, maxOrderDerivative + 1):
    ax = plt.subplot(2, 3, i + 1)
    
    dt = np.diff(tArray)[0]
    derivativeArray = np.gradient(valuesArray, dt)
    plt.plot(tArray[2:-2], derivativeArray[2:-2], 'k-', lw=3)  
    plt.grid('on')
    plt.xlim([0, 516])
    ax.set_xticks(xTicks)
    ax.set_xticklabels([])
    #ax.yaxis.set_major_formatter(FormatStrFormatter('%.1e'))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
    
    plt.ylabel(r'$\mathregular{V^{\left(%d\right)}, мВ/мс^{%d}}$' % (i, i))
    
    if i > 1: # print max value for derivatives of order >= 2
        ax.set_title(r'$\mathregular{\max |V^{\left(%d\right)}| = %.2e \; мВ/мс^{%d}}$' \
                    % (i, np.fabs(derivativeArray[2:-2]).max(), i), fontsize=15, y = 1.15)
    
    if i > 2: # set xlabels the bottom subplots
        ax.set_xticks(xTicks)
        ax.set_xticklabels(xTicks)
        plt.xlabel(r'Время, мс')
    
    tArray = tArray[::4]
    valuesArray = derivativeArray[::4]
    

plt.show()

#Графики аналитических решений при различных k

In [58]:
@nb.jit(nopython=True)
def eps(u, v):
    eps0 = 0.002
    mu1 = 0.2
    mu2 = 0.3
    return eps0 + (mu1*v) / (u + mu2)

@nb.jit(nopython=True)
def rhsU(u, v, k, iStim=0.1):
    return -k*u*(u - a)*(u - 1) - u*v + iStim

@nb.jit(nopython=True)
def rhsV(u, v, k):
    return eps(u, v)*(-v - k*u*(u - a - 1))


@nb.jit((nb.float64[:], nb.int64, nb.float64, nb.int64, \
                        nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], \
                        nb.float64[:], nb.float64, nb.float64, nb.float64), nopython=True)
def CalculateUsingExplicitEuler(time_array, size, dt, counter, u, v, RhsU, RhsV, I_s, k,\
                                u_rest, v_rest):
    u[0] = u_rest
    v[0] = v_rest

    for i in range(1, size):
        RhsU[i - 1] = rhsU(u[i - 1], v[i - 1], k)
        RhsV[i - 1] = rhsV(u[i - 1], v[i - 1], k)

        u[i] = u[i - 1] + dt*RhsU[i - 1]
        v[i] = v[i - 1] + dt*RhsV[i - 1]
 
        
@nb.jit((nb.float64[:], nb.int64, nb.float64, nb.int64, \
                        nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:],\
                       nb.float64[:], nb.float64, nb.float64, nb.float64), nopython=True)
def CalculateUsingSImplicitEuler(time_array, size, dt, counter, u, v, RhsU, RhsV, I_s, k,\
                                 u_rest, v_rest):

    u[0] = u_rest
    v[0] = v_rest
    stepForDer = 1e-2
    
    for i in range(1, size):
        #print('SIE step: complete %.2f' % (i/float(size - 1)))
        RhsU[i - 1] = rhsU(u[i - 1], v[i - 1], k)
        RhsV[i - 1] = rhsV(u[i - 1], v[i - 1], k)

        dRhsUdU = (rhsU(u[i - 1] + stepForDer, v[i - 1], k) \
                                - RhsU[i - 1]) / stepForDer

        v[i] = v[i - 1] + dt*RhsV[i - 1]
        u[i] = u[i - 1] + dt*RhsU[i - 1] / (1 - dt*dRhsUdU)

In [40]:
kArray = [i for i in range(7, 13)]
dt = dtAnalitical
print(dt)
T = 40

time_array = np.arange(0, T + dt, dt)
SIZE = len(time_array)
u = np.zeros(SIZE)
v = np.ones(SIZE)
RhsU = np.zeros(SIZE) # [0. for i in range(size)]
RhsV = np.zeros(SIZE)
u_e = np.zeros(SIZE)
v_e = np.zeros(SIZE) #[0. for i in range(size)]
RhsU_e = np.zeros(SIZE) #[0. for i in range(size)]
RhsV_e = np.zeros(SIZE)
analiticalSolutionsList = []

counter = 0

for k in kArray: 
    print('Iteration #%d' % counter)
    
    CalculateUsingExplicitEuler(time_array, SIZE, dt, counter, u_e, v_e, RhsU_e, RhsV_e, I_s, \
                                                                        k, u_rest, v_rest)
    
    analiticalSolution = np.zeros(len(time_array))
    analiticalSolution[:] = 100*u_e[:] - 80
    MIN, MAX = np.amin(analiticalSolution), np.amax(analiticalSolution)
    amplitude = MAX - MIN
    analiticalSolution[:] -= MIN
    #analiticalSolution = analiticalSolution[::4]

    counter += 1
    analiticalSolutionsList.append(analiticalSolution)
   

# plotting analytical solutions
time_array = 12.9*time_array
#time_array = time_array[::4]
matplotlib.rcParams.update({'font.size': 25})
plt.figure()
for i in range(len(kArray)):
    if kArray[i] == 8:
        plt.plot(time_array, analiticalSolutionsList[i], 'k--', label=r'$\mathbf{k = %d}$' % kArray[i], linewidth=4)
    else:
        plt.plot(time_array, analiticalSolutionsList[i], '-', label=r'$\mathregular{k = %d}$' % kArray[i], linewidth=4)
        plt.legend(loc='best', fontsize=20)
        plt.xlabel(r'Время, мс')
        plt.ylabel(r'V, мВ')
        #plt.ylim([0, 140])
        plt.xlim([0, 516])
        plt.grid('on')
plt.show()

7.62939453125e-05
Iteration #0


Iteration #1


Iteration #2


Iteration #3


Iteration #4


Iteration #5


#Графики ускорений

In [66]:
kArray = [i for i in range(7, 13)]
dtArray = np.asarray([dtAnalitical] + list(float(T)/numBlocksArray[::-1]))
T = 40
TScaled = 12.9 * T
counterInner, counterOuter = 0, 0
speedupRRMS, speedupMaxmod = [], []

for k in kArray:
    timingEE, timingSIE = [], []
    error, error_e = [], []
    counterInner = 0
    for dt in dtArray:

        print ('Iteration #%d' % counterInner)
        time_array = np.arange(0, T + dt, dt)
        time_array_scaled = 12.9 * time_array
           
        dtScaled = 12.9 * dt
        
        SIZE = len(time_array)
        u = np.zeros(SIZE)
        v = np.ones(SIZE)
        RhsU = np.zeros(SIZE) # [0. for i in range(size)]
        RhsV = np.zeros(SIZE)

        u_e = np.zeros(SIZE)
        v_e = np.zeros(SIZE) #[0. for i in range(size)]

        RhsU_e = np.zeros(SIZE) #[0. for i in range(size)]
        RhsV_e = np.zeros(SIZE)

        I_s =10*np.ones(SIZE) #[10. for i in range(size)]



        NUM_LAUNCHES = 20 # N = 3 - used for the article
        startEE = time.clock()
        for i in range(NUM_LAUNCHES):
            CalculateUsingExplicitEuler(time_array, SIZE, dt, counterInner, u_e, v_e, RhsU_e, RhsV_e, I_s, \
                                                                                            k, u_rest, v_rest)
        tEE = (time.clock() - startEE) / NUM_LAUNCHES
        timingEE.append(tEE)

        startSIE = time.clock()
        for i in range(NUM_LAUNCHES):
            CalculateUsingSImplicitEuler(time_array, SIZE, dt, counterInner, u, v, RhsU, RhsV, I_s, \
                                                                                   k, u_rest, v_rest)
        tSIE = (time.clock() - startSIE) / NUM_LAUNCHES
        timingSIE.append(tSIE)


        if counterInner == 0:
            analiticalSolution = np.zeros(len(time_array))
            analiticalSolution[:] = 100*u_e[:] - 80 # scaled to membrane voltage
            MIN, MAX = np.amin(analiticalSolution), np.amax(analiticalSolution)
            amplitude = MAX - MIN
            analiticalSolution[:] -= MIN
            analiticalSolutionFunc = intp.interp1d(time_array_scaled, analiticalSolution)


        u_e[:] = 100*u_e[:] - 80 # scaled to membrane voltage
        u[:] = 100*u[:] - 80 # scaled to membrane voltage

        u[:] -= MIN
        u_e[:] -= MIN 


        errTmp = CalculateNorm(analiticalSolutionFunc, u, TScaled, dtScaled, amplitude)
        errTmp_e = CalculateNorm(analiticalSolutionFunc, u_e, TScaled, dtScaled, amplitude)

        error.append(errTmp)
        error_e.append(errTmp_e)
        
        counterInner += 1


    counterOuter += 1
    error_for_calc = np.asarray(error)
    error_e_for_calc = np.asarray(error_e)

    speedupRRMS.append([intp.interp1d(error_e_for_calc[:, 0], timingEE)(5.) / \
                            intp.interp1d(error_for_calc[:, 0], timingSIE)(5.), \
                            intp.interp1d(error_e_for_calc[:, 0], timingEE)(1.) / \
                            intp.interp1d(error_for_calc[:, 0], timingSIE)(1.)]
                           )

    speedupMaxmod.append([intp.interp1d(error_e_for_calc[:, 1], timingEE)(5.) / \
                              intp.interp1d(error_for_calc[:, 1], timingSIE)(5.), \
                              intp.interp1d(error_e_for_calc[:, 1], timingEE)(1.) / \
                              intp.interp1d(error_for_calc[:, 1], timingSIE)(1.)]
                             )


speedupRRMS = np.asarray(speedupRRMS)
speedupMaxmod = np.asarray(speedupMaxmod)

Iteration #0
Time array, scaled:  [0.00000000e+00 9.84191895e-04 1.96838379e-03 ... 5.15998032e+02
 5.15999016e+02 5.16000000e+02]


Iteration #1
Iteration #2


Iteration #3
Iteration #4
Iteration #5
Iteration #6
Iteration #7
Iteration #8
Iteration #9
Iteration #10
Iteration #11
Iteration #12
Iteration #0


Iteration #1
Iteration #2


Iteration #3
Iteration #4
Iteration #5
Iteration #6
Iteration #7
Iteration #8
Iteration #9
Iteration #10


...

Iteration #1
Iteration #2


Iteration #3
Iteration #4
Iteration #5
Iteration #6
Iteration #7
Iteration #8
Iteration #9
Iteration #10
Iteration #11
Iteration #12
Iteration #0


/Users/quothewhite/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:62: RuntimeWarning: overflow encountered in multiply


Iteration #1
Iteration #2


Iteration #3
Iteration #4
Iteration #5
Iteration #6
Iteration #7
Iteration #8
Iteration #9
Iteration #10
Iteration #11
Iteration #12


In [65]:
plt.figure(figsize=(13, 8.5))
plt.plot(kArray, np.array(speedupRRMS[:, 0]), '-o', label=r'5% RRMS', lw=4, 
         markersize=10)
plt.plot(kArray, speedupRRMS[:, 1], '-o', label=r'1% RRMS', linewidth=4, 
         markersize=10)
plt.plot(kArray, speedupMaxmod[:, 0], '-o', label=r'5% Maxmod', linewidth=4, 
         markersize=10)
plt.plot(kArray, speedupMaxmod[:, 1], '-o', label=r'1% Maxmod', linewidth=4, 
         markersize=10)
plt.legend(loc='best')

plt.xlabel(r'k')
plt.ylabel(r'Ускорение')
#plt.ylim([0.0, 0.4])
plt.grid('on')
plt.show()

In [368]:
speedupRRMSCached, speedupMaxmodCached = speedupRRMS, speedupMaxmod